In [2]:
import os
# Find the latest version of spark 2.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.0'
spark_version = 'spark-3.0.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:9 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:13 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Fetched 252 kB in 2s (110 kB/s)
Reading package lists... Done


In [3]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2020-11-09 03:06:33--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar.1’

postgresql-42.2.16. 100%[===================>] 979.38K  1.01MB/s    in 0.9s    

2020-11-09 03:06:35 (1.01 MB/s) - ‘postgresql-42.2.16.jar.1’ saved [1002883/1002883]



In [4]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

### Load Amazon Data into Spark DataFrame

In [5]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Books_v1_01.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   22480053|R28HBXXO1UEVJT|0843952016|      34858117|          The Rising|           Books|          5|            0|          0|   N|                N|Great Twist on Zo...|I've known about ...| 2012-05-03|
|         US|   44244451| RZKRFS2UUMFFU|031088926X|     676347131|Sticky Faith Teen...|           Books|          5|    

### Create DataFrames to match tables

In [6]:
from pyspark.sql.functions import to_date
# Read in the Review dataset as a DataFrame
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   22480053|R28HBXXO1UEVJT|0843952016|      34858117|          The Rising|           Books|          5|            0|          0|   N|                N|Great Twist on Zo...|I've known about ...| 2012-05-03|
|         US|   44244451| RZKRFS2UUMFFU|031088926X|     676347131|Sticky Faith Teen...|           Books|          5|    

In [7]:
# Create the vine_table. DataFrame
vine_df = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R28HBXXO1UEVJT|          5|            0|          0|   N|                N|
| RZKRFS2UUMFFU|          5|           15|         15|   N|                Y|
|R2WAU9MD9K6JQA|          3|            6|          8|   N|                N|
|R36SCTKYTVPZPC|          5|           10|         11|   N|                Y|
|R10BM6JUOJX27Q|          3|            0|          0|   Y|                N|
| RCLZ5OKZNUSY4|          5|            0|          0|   N|                Y|
|R1S65DJYEI89G4|          4|            8|         17|   N|                N|
|R3KQYBQOLYDETV|          4|            2|          2|   N|                N|
|R3QV8K7CSU8K2W|          5|            0|          0|   N|                N|
|R3W5A1WUGO5VQ0|          4|            0|          1|   N|     

### Vine Table Data Analysis


In [8]:
# retrieve rows where total_votes count is equal to or greater than 20
filtered_total_votes = vine_df.filter("total_votes>=20").select(['review_id', 'star_rating', 'helpful_votes', 'vine', 'verified_purchase', 'total_votes'])
filtered_total_votes.show()

+--------------+-----------+-------------+----+-----------------+-----------+
|     review_id|star_rating|helpful_votes|vine|verified_purchase|total_votes|
+--------------+-----------+-------------+----+-----------------+-----------+
|R18VIM840CEFRP|          1|           16|   N|                N|        105|
|R14PMAJTY0EAAT|          4|          135|   N|                Y|        142|
|R1363VA3TPNLVB|          5|          370|   N|                Y|        388|
| RBQZC5A3TSWT5|          5|           11|   N|                Y|         22|
| RW00TDPV9U93E|          1|           35|   N|                N|         76|
|R3OW0AIVLEDIQ7|          3|           99|   N|                N|        121|
|R3DTESO4FUAPKQ|          5|           90|   N|                N|        102|
|R18I0XBYWFSR5T|          1|           16|   N|                N|         50|
|R1VSQ3QK30VDI0|          5|           23|   N|                N|         35|
|R3NEJBF4FXMVUU|          5|           87|   N|                Y

In [9]:
# filter Dataframe to retrieve all the rows where the number helpful_votes divided by total_votes is equal to or great than 50%
greater_than_50 = filtered_total_votes.filter("helpful_votes/total_votes>=0.5")
greater_than_50.show()

+--------------+-----------+-------------+----+-----------------+-----------+
|     review_id|star_rating|helpful_votes|vine|verified_purchase|total_votes|
+--------------+-----------+-------------+----+-----------------+-----------+
|R14PMAJTY0EAAT|          4|          135|   N|                Y|        142|
|R1363VA3TPNLVB|          5|          370|   N|                Y|        388|
| RBQZC5A3TSWT5|          5|           11|   N|                Y|         22|
|R3OW0AIVLEDIQ7|          3|           99|   N|                N|        121|
|R3DTESO4FUAPKQ|          5|           90|   N|                N|        102|
|R1VSQ3QK30VDI0|          5|           23|   N|                N|         35|
|R3NEJBF4FXMVUU|          5|           87|   N|                Y|         90|
|R34ZTQTHWRHT5R|          5|           18|   N|                Y|         20|
|R367X555006ROC|          1|           39|   N|                N|         44|
| R4NCMAZCRAXHT|          3|           19|   N|                N

In [10]:
#show percentage of helpful votes
greater_than_50_percents = greater_than_50.withColumn('percentage_helpful_vote', greater_than_50.helpful_votes / greater_than_50.total_votes)
greater_than_50_percents.show()

+--------------+-----------+-------------+----+-----------------+-----------+-----------------------+
|     review_id|star_rating|helpful_votes|vine|verified_purchase|total_votes|percentage_helpful_vote|
+--------------+-----------+-------------+----+-----------------+-----------+-----------------------+
|R14PMAJTY0EAAT|          4|          135|   N|                Y|        142|     0.9507042253521126|
|R1363VA3TPNLVB|          5|          370|   N|                Y|        388|     0.9536082474226805|
| RBQZC5A3TSWT5|          5|           11|   N|                Y|         22|                    0.5|
|R3OW0AIVLEDIQ7|          3|           99|   N|                N|        121|     0.8181818181818182|
|R3DTESO4FUAPKQ|          5|           90|   N|                N|        102|     0.8823529411764706|
|R1VSQ3QK30VDI0|          5|           23|   N|                N|         35|     0.6571428571428571|
|R3NEJBF4FXMVUU|          5|           87|   N|                Y|         90|     

In [11]:
# filter dataframe and create new DataFrame or table that retrieves all the rows where a review was written as part of the vine program (paid), vine == 'Y'
vine_program_df = greater_than_50_percents.filter("vine=='Y'")
vine_program_df.show()

+--------------+-----------+-------------+----+-----------------+-----------+-----------------------+
|     review_id|star_rating|helpful_votes|vine|verified_purchase|total_votes|percentage_helpful_vote|
+--------------+-----------+-------------+----+-----------------+-----------+-----------------------+
|R2Z8083V8ZFQVZ|          3|           82|   Y|                N|         87|     0.9425287356321839|
| RK25TAO4GGS5G|          3|           22|   Y|                N|         24|     0.9166666666666666|
|R2SJQG3C6KY0M8|          3|           18|   Y|                N|         20|                    0.9|
|R12CHC0CB2WASU|          5|           43|   Y|                N|         48|     0.8958333333333334|
|R2RY328TIDXMTE|          2|           50|   Y|                N|         61|      0.819672131147541|
|R3K8OQU4PEP8W7|          4|           40|   Y|                N|         46|     0.8695652173913043|
|R1JI1A7JT99538|          3|           28|   Y|                N|         40|     

In [12]:
# retrieve all the rows where the review was not part of the Vine program (unpaid), vine == 'N'
not_vine_program_df = greater_than_50_percents.filter("vine=='N'")
not_vine_program_df.show()

+--------------+-----------+-------------+----+-----------------+-----------+-----------------------+
|     review_id|star_rating|helpful_votes|vine|verified_purchase|total_votes|percentage_helpful_vote|
+--------------+-----------+-------------+----+-----------------+-----------+-----------------------+
|R14PMAJTY0EAAT|          4|          135|   N|                Y|        142|     0.9507042253521126|
|R1363VA3TPNLVB|          5|          370|   N|                Y|        388|     0.9536082474226805|
| RBQZC5A3TSWT5|          5|           11|   N|                Y|         22|                    0.5|
|R3OW0AIVLEDIQ7|          3|           99|   N|                N|        121|     0.8181818181818182|
|R3DTESO4FUAPKQ|          5|           90|   N|                N|        102|     0.8823529411764706|
|R1VSQ3QK30VDI0|          5|           23|   N|                N|         35|     0.6571428571428571|
|R3NEJBF4FXMVUU|          5|           87|   N|                Y|         90|     

In [18]:
# Determine the total number of reviews (paid and unpaid)
paid_count = vine_program_df.count()
unpaid_count = not_vine_program_df.count()

print(paid_count)
print(unpaid_count)

4781
332395


In [19]:
# the number of 5-star reviews, and the percentage of 5-star reviews for the two types of review (paid).
five_star_count = vine_program_df[vine_program_df.star_rating==5].count()
print(five_star_count)

percent_five_star_paid = five_star_count/paid_count
print(percent_five_star_paid)

1604
0.33549466638778497


In [20]:
# the number of 5-star reviews, and the percentage of 5-star reviews for the two types of review (unpaid).
five_star_count_unpaid = not_vine_program_df[not_vine_program_df.star_rating==5].count()
print(five_star_count_unpaid)

percent_five_star_unpaid = five_star_count_unpaid/unpaid_count
print(percent_five_star_unpaid)

168800
0.5078295401555378
